# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [3]:
#Your code here
import pandas as pd

# Step 1: Load the dataset
df = pd.read_csv('homepage_actions.csv')

# Step 2: Exploratory Data Analysis (EDA)
# Check the first few rows of the dataframe
print(df.head())

# Check the structure of the dataframe
print(df.info())

# Investigate the 'id' column
# How many viewers also clicked?
viewers_clicked = df[df['action'] == 'click']['id'].nunique()
print("Number of viewers who clicked:", viewers_clicked)

# Are there any anomalies with the data? (e.g., clicks without views)
anomalies = df[df['action'] == 'click']['id'].isin(df[df['action'] == 'view']['id'])
print("Anomalies in the data (clicks without views):", anomalies.any())

# Is there any overlap between the control and experiment groups?
overlap = df.groupby('group')['id'].nunique()
print("Overlap between control and experiment groups:\n", overlap)

# Step 3: Experimental Design
# Based on the overlap, decide how to account for it in your analysis

# Step 4: Analysis and Conclusion
# Proceed with analyzing the results and drawing conclusions


                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None
Number of viewers who clicked: 1860
Anomalies in the data (clicks without views): True
Overlap between control and experiment groups:
 group
control       3332
experiment    2996
Name: id, dtype: int64


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [4]:
#Your code here
from scipy import stats

# Separate data for control and experimental groups
control_data = df[df['group'] == 'control']['action']
experimental_data = df[df['group'] == 'experiment']['action']

# Perform two-sample t-test
t_statistic, p_value = stats.ttest_ind(control_data == 'click', experimental_data == 'click')

# Print the results
print("t-statistic:", t_statistic)
print("p-value:", p_value)

# Compare p-value with significance level
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis. The experimental homepage is more effective.")
else:
    print("Fail to reject the null hypothesis. There is insufficient evidence to conclude that the experimental homepage is more effective.")


t-statistic: -1.9334751824865355
p-value: 0.05321212418167477
Fail to reject the null hypothesis. There is insufficient evidence to conclude that the experimental homepage is more effective.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [8]:
# Separate data for control and experimental groups
control_data = df[df['group'] == 'control']
experimental_data = df[df['group'] == 'experiment']

# Calculate the control group click-through rate
control_ctr = control_data['action'].eq('click').mean()

# Calculate the experimental group size
experimental_group_size = len(experimental_data)

# Calculate the expected number of clicks for the experimental group
expected_clicks = experimental_group_size * control_ctr

# Print the result
print("Expected number of clicks for the experimental group:", expected_clicks)


Expected number of clicks for the experimental group: 857.6848030018762


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [10]:
#Your code here
import numpy as np

# Actual number of clicks in the experimental group
actual_clicks = experimental_data['action'].eq('click').sum()

# Calculate the standard deviation
std_dev = np.sqrt(experimental_group_size * control_ctr * (1 - control_ctr))

# Calculate the number of standard deviations
z_score = (actual_clicks - expected_clicks) / std_dev

# Print the result
print("Number of standard deviations from the estimate:", z_score)


Number of standard deviations from the estimate: 2.71607302278784


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [11]:
#Your code here
from scipy.stats import norm

# Calculate the p-value using the cumulative distribution function (CDF)
p_value = norm.cdf(z_score)

# Print the result
print("P-value:", p_value)


P-value: 0.9966969327240734


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: The calculated p-value of approximately 0.997 indicates that the observed data is very likely under the null hypothesis, which suggests that there is no significant difference in click-through rates between the experimental and control groups. This result aligns with the previous statistical test, which also indicated that the experimental homepage was not significantly more effective than the control group's homepage.

In summary, both the previous statistical test and the calculation based on the z-score and p-value suggest that there is no evidence to reject the null hypothesis, implying that the experimental homepage did not lead to a significant increase in click-through rates compared to the control group.








## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.